# 0. 패키지 임포트

In [2]:
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_upstage import UpstageEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from decouple import config
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\Admin\anaconda3\envs\llm-lab\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. 기본 세팅

In [16]:
load_dotenv()

True

In [17]:
pc = Pinecone()

In [18]:
# embedding = OpenAIEmbeddings(model="text-embedding-3-large")
embedding = UpstageEmbeddings(model="solar-embedding-1-large")
index_name = config("PINECONE_INDEX")


In [19]:
database = PineconeVectorStore(
    embedding=embedding, 
    index_name=index_name
)

In [20]:
retriever = database.as_retriever(search_kwargs={"k": 3})
prompt = hub.pull('rlm/rag-prompt')
llm = ChatOpenAI(model="gpt-4.1-nano")

In [21]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = retriever,
    chain_type_kwargs={"prompt": prompt},
)

# 2. 선다형 문제로 검증 실행

In [23]:
df = pd.read_csv("법령문제_금융은행보험.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,보기1,보기2,보기3,보기4,정답,해설
2033,선주상호보험조합법(법률)(제20436호)(20250131),사지선다형,중,조합원이 되려는 자가 조합원 가입을 거부당할 수 있는 경우는 무엇인가?,A. 조합원이 될 자격이 없는 경우,B. 조합의 정관에 따라,C. 조합원이 아닌 자를 피보험자로 하는 경우,D. 조합원이 탈퇴한 경우,1,제19조에 따르면 조합원이 될 자격이 있는 자가 조합원이 되려는 경우 조합은 정당한...


In [24]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):
    sample = df.loc[i]
    question = """아래 문제를 보고 답을 고르시오.
만약 보기 중 3번째 보기가 정답이라면 3 이라고 출력하시오.
보기가 A,B,C,D 라면 1,2,3,4 라고 생각하고 답변하시오.
오직 숫자만 출력하시오.
출력 예시: 3
"""
    question += sample['문제내용'] + '\n'

    for i in range(1,5):
        col = '보기' + str(i)
        question += sample[col] + '\n'

    message = qa_chain.invoke({"query":question})['result']
    if message == str(sample['정답']):
        success += 1
    else:
        fail += 1
    gpt.append(message)
    act.append( str(sample['정답']))

In [25]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 34
오답 갯수 : 16
정확도 : 0.68
챗봇에 대답:  ['1', '2', '2', '2', '3', '2', '4', '2', '5', '2', '3', '2', '2', '3', '2', '3', '1', '2', '3', '2', '2', '2', '2', '1', '2', '2', '3', '4', '3', '2', '2', '2', '3', '2', '2', '2', '1', '3', '3', '2', '2', '3', '2', '2', '3', '2', '2', '3', '3', '6']
실제 답    :  ['1', '3', '2', '3', '3', '4', '4', '1', '2', '2', '3', '2', '2', '2', '3', '2', '1', '2', '3', '2', '2', '2', '2', '1', '2', '2', '3', '1', '1', '2', '2', '2', '1', '2', '2', '2', '1', '3', '1', '2', '2', '3', '2', '1', '4', '2', '2', '3', '1', '3']


# 3. 단답형 문제로 검증 실행

In [26]:
df = pd.read_csv("단답형_문제.csv")
df.sample()

,법령명,문제유형,난이도,문제내용,정답,해설
137,금융혁신지원 특별법(법률)(제20054호)(20241017),단답형,하,금융위원회가 혁신금융서비스 지정 취소를 할 수 있는 경우는?,거짓으로 지정받은 경우,제7조에 따르면 거짓으로 지정받은 경우 취소할 수 있다.


In [27]:
results = []
SIMILARITY_THRESHOLD = 0.7

In [32]:
success, fail = 0, 0
gpt = []
act = []
for i in range(50):

    question = """아래 문제를 보고 짧고 간단하게 답을 단답식으로 제출하시오.
    좋은 답변들 예시 : ['대주주와의 거래', '지배구조법 대주주', '금융위원회', '제12조', '5%']
    '대통령령으로 정하는 기간은 3개월이다.' -> '3개월' 
    '개인금융채권의 원금이 5천만원 이상이다.'-> '5천만원'
    이와같이 사족없이 30자 이내로 핵심 단어의 단답형으로 답변하시오.
    """

    sample = df.loc[i]
    question += sample['문제내용']
    answer = sample['정답']


    
    message = qa_chain.invoke({"query": question})['result']

    ground_truth_embedding = embedding.embed_query(answer)
    prediction_embedding = embedding.embed_query(message)
    
    score = cosine_similarity(
        np.array(ground_truth_embedding).reshape(1, -1),
        np.array(prediction_embedding).reshape(1, -1)
    )[0][0]
    
    if score >= SIMILARITY_THRESHOLD:
        success += 1
    else:
        fail += 1
        
    gpt.append(message)
    act.append(answer)

APIConnectionError: Connection error.

In [33]:
print('정답 갯수 :', success)
print('오답 갯수 :', fail)
print('정확도 :', (success)/(success+fail))
print('챗봇에 대답: ', gpt)
print('실제 답    : ', act)

정답 갯수 : 1
오답 갯수 : 3
정확도 : 0.25
챗봇에 대답:  ['개인금융채권이란 법 제11조제1항에서 대통령령으로 정하는 채권을 말한다.', '개인금융채권 연체, 채무조정, 주소불명', '기간 1년', '1만분의 10  \n50억원']
실제 답    :  ['법 제3조제1항 각 호의 권리로 담보된 개인금융채권', '연체기간이나 연체횟수를 초과한 경우', '3개월', '5천만원']
